In [1]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/multi_agent_sim")
from multi_agent_sim import MultiAgentSim

In [2]:

social_network_params_high_var = {
    'location_vec_dim': 2,
    'network_gamma': 3,
    'daily_contacts_distn_type': 'negative_binomial',
    'neg_bin_r': 2,
    'neg_bin_p_hyperparams': (2,3),
}

n_agents = 1000
expected_total_outside_infs = 5
time_horizon = 7 * 12
main_params = {
    'n_agents': n_agents,
    'use_contact_tracing': True,
    'use_adaptive_testing': True,
    'use_surveillance_testing': True
}


infection_dynamics_params = {
    'forward_gen_alpha': 8,
    'forward_gen_beta_hyperparams': (5,5),
    'detectability_curve_type': 'optimistic',
    'self_reporting_multiplier': 0.8,
    'self_reporting_delay': 3,
    'init_infection_rate': 0,
    'use_deterministic_infection_counts': True,
    'disc_gamma_transmission_mult': 1,
    'outside_infection_rate': expected_total_outside_infs / (n_agents * time_horizon)
}

ct_params = {
    'ct_recall_window': 4,
    'ct_delay_distribution': [1/3,1/3,1/3], # uniform over 0, 1, 2 days delay
    'ct_recall_rate': 0.5,
    'min_delay_between_traces': 14
}

at_params = {
    'at_delay_distribution': [1/3,1/3,1/3],
    'at_net_size_contact_multiplier': 10,
    'at_recall_rate': 0.9
}

st_params = {
    'st_testing_window': 3,
    'st_missed_test_rate': 0.1
}

def init_high_var_sim(R0):
    social_network_params_high_var['neg_bin_r'] = R0 / 0.9
    return MultiAgentSim(main_params, infection_dynamics_params, social_network_params_high_var, 
                         ct_params, at_params, st_params)

neg_bin_r = 2
social_network_params_low_var = {
    'location_vec_dim': 2,
    'network_gamma': 3,
    'daily_contacts_distn_type': 'negative_binomial',
    'neg_bin_r': neg_bin_r,
    'neg_bin_p_hyperparams': (5,6)
}

# avg close contacts / day = 2 * 5 / (6-1)

def init_low_var_sim(R0):
    infection_dynamics_params['disc_gamma_transmission_mult'] = R0 / neg_bin_r
    return MultiAgentSim(main_params, infection_dynamics_params, social_network_params_low_var, 
                         ct_params, at_params, st_params)

In [3]:
def sample_trajectory(R0, at_multiplier, st_testing_window, 
                      use_adaptive_testing = True, use_surveillance_testing = True):
    st_params['st_testing_window'] = st_testing_window
    at_params['at_net_size_contact_multiplier'] = at_multiplier
    main_params['use_adaptive_testing'] = use_adaptive_testing
    main_params['use_surveillance_testing'] = use_surveillance_testing
    sim = init_low_var_sim(R0)
    infection_counts = [len(sim.infection.get_cum_infected_agent_ids())]
    init_infection_counts = infection_counts[0]
    for _ in range(time_horizon):
        sim.step()
        infection_counts.append(len(sim.infection.get_cum_infected_agent_ids()))
    total_tests = sim.get_total_tests()
    return total_tests, infection_counts, init_infection_counts, infection_counts[-1]

In [4]:
from multiprocessing import Process
import pickle


def sim_target_f(R0, at_mult, st_window, use_at, use_st, ntrajectories, pickle_file_loc):
    results = [sample_trajectory(R0, at_mult, st_window, use_at, use_st) for _ in range(ntrajectories)]
    pickle.dump(results, open(pickle_file_loc, "wb"))

def run_sims_new_process(R0, at_mult, st_window, use_at, use_st, ntrajectories, pickle_file_loc):
    p = Process(target = sim_target_f, args = (R0, at_mult, st_window, use_at, use_st, ntrajectories, pickle_file_loc))
    p.start()
    return p




In [ ]:
R0s_to_try = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

at_mults = [4,8,12,16]

st_freqs = [4,7,10,14, 18, 21]

output_dir = "jan_07_multiagent_sims/"

def run_many_simulations(R0s_to_try, at_mults, st_freqs, output_dir):

    adaptive_test_policies = []

    surveillance_test_policies = []

    no_use = 1

    for st_freq in st_freqs:
        surveillance_test_policies.append((no_use, st_freq, False, True))
        for at_mult in at_mults:
            adaptive_test_policies.append((at_mult, st_freq, True, True))


    ntrajectories = 250

    sim_ps = {}

    from datetime import datetime

    all_policies = adaptive_test_policies + surveillance_test_policies



    for R0 in R0s_to_try:
        for policy in all_policies:
            at_mult, st_window, use_at, use_st = policy
            sim_ps[(R0, policy)] = run_sims_new_process(R0, at_mult, st_window, 
                                                        use_at, use_st, ntrajectories, 
                                                        output_dir + str((R0, policy)) + ".pickle")

    print("launched {} processes".format(sim_ps.values()))
    for p in sim_ps.values():
        p.join()
        
run_many_simulations(R0s_to_try, at_mults, st_freqs, output_dir)

launched dict_values([<Process(Process-271, started)>, <Process(Process-272, started)>, <Process(Process-273, started)>, <Process(Process-274, started)>, <Process(Process-275, started)>, <Process(Process-276, started)>, <Process(Process-277, started)>, <Process(Process-278, started)>, <Process(Process-279, started)>, <Process(Process-280, started)>, <Process(Process-281, started)>, <Process(Process-282, started)>, <Process(Process-283, started)>, <Process(Process-284, started)>, <Process(Process-285, started)>, <Process(Process-286, started)>, <Process(Process-287, started)>, <Process(Process-288, started)>, <Process(Process-289, started)>, <Process(Process-290, started)>, <Process(Process-291, started)>, <Process(Process-292, started)>, <Process(Process-293, started)>, <Process(Process-294, started)>, <Process(Process-295, started)>, <Process(Process-296, started)>, <Process(Process-297, started)>, <Process(Process-298, started)>, <Process(Process-299, started)>, <Process(Process-300,

Process Process-440:
Traceback (most recent call last):
  File "/home/jmc678/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/jmc678/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-c181455b1441>", line 6, in sim_target_f
    results = [sample_trajectory(R0, at_mult, st_window, use_at, use_st) for _ in range(ntrajectories)]
  File "<ipython-input-4-c181455b1441>", line 6, in <listcomp>
    results = [sample_trajectory(R0, at_mult, st_window, use_at, use_st) for _ in range(ntrajectories)]
  File "/home/jmc678/covid_data/group-testing/src/multi_agent_sim/multi_agent_sim.py", line 86, in step
    self.at.trigger_adaptive_test(agent_id, self.curr_day)
  File "<ipython-input-3-80d590b9f95b>", line 11, in sample_trajectory
    sim.step()
  File "/home/jmc678/covid_data/group-testing/src/multi_agent_sim/adaptive_testing.py", line 34, in trigger_ad